## Hello, this is my solution of Cardiac Risk Prediction hackathon organized by Machinehack Analytics India Magazine. This data can be utilised for predicting the analysis related to risk of cardiac arrest.

### Below solution can be implemented for medical purposes

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier, BaggingClassifier, VotingClassifier

In [ ]:
train = pd.read_csv(r'/kaggle/input/cardiac-risk-prediction/Train.csv')
test = pd.read_csv(r'/kaggle/input/cardiac-risk-prediction/Test.csv')
sample = pd.read_excel(r'/kaggle/input/cardiac-risk-prediction/Sample_Submission.xlsx')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
for c in train.columns :
    print(train[c].value_counts())

In [ ]:
sns.countplot(train.UnderRisk)

# ANALYSIS

In [ ]:
for c in train.columns[:-1]:
    print(train.groupby(c)['UnderRisk'].value_counts(),'\n\n')

In [ ]:
ind_no = []
train = train.groupby('Family_history').get_group(1).reset_index(drop = True)
d = test[test.Family_history==0].index
ind_no.extend(d)
del train['Family_history']
del test['Family_history']
test.drop(index = d, inplace = True)
len(ind_no)

In [ ]:
train = train.groupby('CABG_history').get_group(0).reset_index(drop = True)
d = test[test.CABG_history==1].index.tolist()
del train['CABG_history']
del test['CABG_history']
ind_no.extend(d)
test.drop(index = d, inplace = True)
len(ind_no)

### Gen 1 -> Female

In [ ]:
train[train.History_of_preeclampsia==1]

In [ ]:
train.drop(index=304, inplace = True)
train.reset_index(drop = True, inplace = True)

In [ ]:
train = train.groupby('History_of_preeclampsia').get_group(0).reset_index(drop = True)
d = test[test.History_of_preeclampsia==1].index.tolist()
del train['History_of_preeclampsia']
del test['History_of_preeclampsia']
ind_no.extend(d)
test.drop(index = d, inplace = True)
len(ind_no)

In [ ]:
train = train.groupby('Chain_smoker').get_group(0).reset_index(drop = True)
d = test[test.Chain_smoker==1].index.tolist()
del train['Chain_smoker']
del test['Chain_smoker']
ind_no.extend(d)
test.drop(index = d, inplace = True)
len(ind_no)

In [ ]:
del train['Metabolic_syndrome']
del test['Metabolic_syndrome']

In [ ]:
train.head(1)

In [ ]:
train['disease'] = train.Gender.astype('str')+train.Consumes_other_tobacco_products.astype('str')+train.Use_of_stimulant_drugs.astype('str')+train['HighBP'].astype('str')+train.Obese.astype('str')+train.Diabetes.astype('str')+train.Respiratory_illness.astype('str')
test['disease'] = test.Gender.astype('str')+test.Consumes_other_tobacco_products.astype('str')+test.Use_of_stimulant_drugs.astype('str')+test['HighBP'].astype('str')+test.Obese.astype('str')+test.Diabetes.astype('str')+test.Respiratory_illness.astype('str')

In [ ]:
train['any_drug'] = train.Consumes_other_tobacco_products + train.Use_of_stimulant_drugs
test['any_drug'] = test.Consumes_other_tobacco_products + test.Use_of_stimulant_drugs

In [ ]:
tr = set(train.disease).difference(set(test.disease))
te = set(test.disease).difference(set(train.disease))

train.disease = train.disease.apply(lambda x: 'other' if x in tr else x).astype('category')
test.disease = test.disease.apply(lambda x: 'other' if x in te else x).astype('category')

In [ ]:
val = {'no':['2010000','2001100', '2111000', '1100111', '1000101','1111000'],
         'yes':['1101110','0110000']}

In [ ]:
d = list(test[test.disease.apply(lambda x: x in val['no'])].index)
test.drop(index = d, inplace = True)
ind_no.extend(d)

In [ ]:
ind_yes = []
d = list(test[test.disease.apply(lambda x: x in val['yes'])].index)
test.drop(index = d, inplace = True)
ind_yes.extend(d)

In [ ]:
for i,j in enumerate(train.columns):
    for c in train.columns[i+1:]:
        print('Columns Fixed v/s Variable:\t', j,c)
        print(train.groupby([j,c])['UnderRisk'].value_counts(),'\n\n')

Gender  Consumes_other_tobacco_products  UnderRisk
2       0                                no            11
                                         yes            1
                                         
Consumes_other_tobacco_products  Metabolic_syndrome  UnderRisk
0                                0                   no            43
                                                     yes           21
                                 1                   no             2
1                                0                   no           470
                                                     yes          161
                                 1                   yes            2
                                 
Consumes_other_tobacco_products  Respiratory_illness  UnderRisk
0                                0                    no            42
                                                      yes           21
                                 1                    no             3

Obese  Respiratory_illness  UnderRisk
0      0                    no            24
                            yes           11
       1                    yes            2

Use_of_stimulant_drugs  Respiratory_illness  UnderRisk
0                       0                    no           474
                                             yes          162
                        1                    no            16
                                             yes            9
1                       0                    no            25
                                             yes           11
                        1                    yes            2

Columns Fixed v/s Variable:	 Consumes_other_tobacco_products Respiratory_illness
Consumes_other_tobacco_products  Respiratory_illness  UnderRisk
0                                0                    no            65
                                                      yes           22
                                 1                    no             4
1                                0                    no           516
                                                      yes          157
                                 1                    no            14
                                                      yes           11

In [ ]:
train.shape

In [ ]:
tr_dummies = pd.DataFrame(pd.get_dummies(train[['Gender','disease','any_drug']].astype('category')))
te_dummies = pd.DataFrame(pd.get_dummies(test[['Gender','disease','any_drug']].astype('category')))

train = pd.concat([train,tr_dummies],axis=1)
test = pd.concat([test,te_dummies],axis=1)

train.drop(columns=['Gender','disease','any_drug'],inplace=True)
test.drop(columns=['Gender','disease','any_drug'],inplace=True)

In [ ]:
label = train.UnderRisk
label = label.apply(lambda x : 1 if (x=='yes') else 0)
train.drop(columns=['UnderRisk'],inplace=True)

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

sm = SMOTETomek(ratio = 'auto')
x,y = sm.fit_sample(train, label)
train = pd.DataFrame(x, columns = train.columns)
label = pd.DataFrame(y)

In [ ]:
515+184

In [ ]:
label.shape

In [ ]:
from keras.layers import *
from keras.utils import to_categorical
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(32, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
#model.add(Dense(16, activation = 'relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss = 'binary_crossentropy', metrics=['accuracy'], optimizer = 'adam')

In [ ]:
model.fit(train.values, to_categorical(label,2), batch_size=128, epochs = 100, validation_split=0.2, shuffle=True)

In [ ]:
def model(m, split ,train, label, test, random, need = False):
    te = []
    strat = StratifiedKFold(n_splits= split, shuffle=True, random_state = random)
    for tr_index,te_index in strat.split(train,label):
        
        xtrain,xval = train.loc[tr_index,:],train.loc[te_index,:]
        ytrain,yval = label.loc[tr_index], label.loc[te_index]
        
        m.fit(xtrain,ytrain)
        tr_pred = m.predict_proba(xtrain)
        te_pred = m.predict_proba(xval)
        
        print('Training Loss :{}                                    Testing Loss : {}'.format(log_loss(ytrain,tr_pred),log_loss(yval,te_pred)))
        if need :
            te.append(pd.DataFrame(m.predict_proba(test), index = test.index))
    return(te)    

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
model(RandomForestClassifier(random_state=12312),5,train,label,test,2344)

In [ ]:
model(DecisionTreeClassifier(random_state=124),5,train,label,test,1323)

In [ ]:
result = model(LogisticRegression(random_state=9994),5,train,label,test,245993, need = True)

In [ ]:
model(GradientBoostingClassifier(random_state = 65444),5,train,label,test,331022)

In [ ]:
result = model(SVC(probability=True, random_state=5410, kernel = 'rbf', C = 0.1),5,train,label,test,331022, True)

In [ ]:
import lightgbm as lgb
import xgboost as xgb
result = model(lgb.LGBMClassifier(random_state=2993),5,train,label,test,34995, need = True)

In [ ]:
result = model(xgb.XGBClassifier(random_state=2304),5,train,label,test,34005, True)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

xtr, xval, ytr,yval = train_test_split(train, label, test_size=0.2, random_state = 123)
param = [{'num_leaves':[5,10,15,20,25,30,40,50,65],
         'lambda_l1':[0.1,0.3,0.6,0.8],
         'lambda_l2':[0.1,0.4,1,2,5,6,8,10]}]
gs = GridSearchCV(param_grid=param, cv=3, estimator=lgb.LGBMClassifier(random_state = 31), scoring='neg_log_loss', verbose=1,n_jobs=-1)
gs.fit(xtr, ytr)

In [ ]:
gs.best_score_, gs.best_estimator_

In [ ]:
est_1 = gs.best_estimator_

In [ ]:
xgb.XGBClassifier()

In [ ]:
param = [{'max_depth':[2,3,4,5,7,9,12],
         'colsample_bytree':[0.1,0.3,0.6,0.8],
         'subsample':[0.1,0.4,0.6,0.7,0.8,0.9]}]
gs = GridSearchCV(param_grid=param, cv=3, estimator=xgb.XGBClassifier(random_state = 311), scoring='neg_log_loss', verbose=1,n_jobs=-1)
gs.fit(xtr, ytr)

In [ ]:
gs.best_score_, gs.best_estimator_

In [ ]:
est_2 = gs.best_estimator_

In [ ]:
LogisticRegression()

In [ ]:
param = [{'C':[0.01,0.1,1,2,3],
         'l1_ratio':[0.1,0.3,0.6,0.8],
         'max_iter':[10,50,100,200,300,500,1000],
          'warm_start':[True, False]}]
gs = GridSearchCV(param_grid=param, cv=3, estimator=LogisticRegression(random_state=990110), scoring='neg_log_loss', verbose=1,n_jobs=-1)
gs.fit(xtr, ytr)

In [ ]:
gs.best_score_, gs.best_estimator_

In [ ]:
est_3 = gs.best_estimator_

In [ ]:
result = model(StackingClassifier(final_estimator = LogisticRegression(random_state=12113),
                        estimators = [('lg', est_1),
                                      ('xg', est_2),
                                      ('lr', est_3)]),5,train,label,test,511200,True)

In [ ]:
model(VotingClassifier(estimators = [('lg', lgb.LGBMClassifier(random_state=2)),
                                      ('xg', xgb.XGBClassifier(random_state=2)),
                                      ('lr',LogisticRegression(random_state=2))], voting = 'soft'),5,train,label,test,22)

In [ ]:
temp = pd.DataFrame({0: 1.0, 1:0.0}, index = ind_no)
temp3 = pd.DataFrame({0: 0.0, 1:1.0}, index = ind_yes)

In [ ]:
result = pd.DataFrame({0:(result[0][0]+result[1][0]+result[2][0]+result[3][0]+result[4][0])/5,
             1:(result[0][1]+result[1][1]+result[2][1]+result[3][1]+result[4][1])/5}, index= result[0].index)

In [ ]:
result = pd.concat([result,temp, temp3], axis = 0).sort_index()

In [ ]:
result.shape

In [ ]:
sample.shape, result.shape

In [ ]:
result.columns= sample.columns

In [ ]:
result

In [ ]:
result.to_excel('stack_4.xlsx',index=False)